In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
## Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

In [ ]:
## Preprocess the data
### Drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

In [ ]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
## Onehot encode 'Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

In [ ]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [ ]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

In [9]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [ ]:
data.head()

In [11]:
## DiVide the dataset into indepent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [ ]:
X_train

In [13]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [ ]:
data

In [ ]:
import tensorflow as tf
print(tf.__version__)

### ANN Implementation

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
## Define the model

model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'), ## HL2 connected with HL1
    Dense(16, activation='relu'), ## HL3 connected with HL2
    Dense(1, activation='sigmoid') ## Output layer connected with HL3
])

model.summary()


In [25]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)  # Define the optimizer with a learning rate
## Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
##Set up the TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [30]:
## Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
### Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=50, 
                    callbacks=[tensorboard_callback, early_stopping]
                    )

In [ ]:
model.save('model.h5')

In [34]:
## Load TensorBoard logs
%load_ext tensorboard

In [40]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 64873), started 0:03:11 ago. (Use '!kill 64873' to kill it.)